<a href="https://colab.research.google.com/github/a-r-t-i-f-i-c-i-a-l/ml4qs/blob/master/ML4QS_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://wifo5-14.informatik.uni-mannheim.de/sensor/dataset/dailylog2016/dailylog2016_dataset.zip

--2021-06-15 16:25:08--  http://wifo5-14.informatik.uni-mannheim.de/sensor/dataset/dailylog2016/dailylog2016_dataset.zip
Resolving wifo5-14.informatik.uni-mannheim.de (wifo5-14.informatik.uni-mannheim.de)... 134.155.95.56
Connecting to wifo5-14.informatik.uni-mannheim.de (wifo5-14.informatik.uni-mannheim.de)|134.155.95.56|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4944490674 (4.6G) [application/zip]
Saving to: ‘dailylog2016_dataset.zip.1’

dailylog2016_datase 100%[===================>]   4.60G  27.6MB/s    in 2m 52s  

2021-06-15 16:28:00 (27.5 MB/s) - ‘dailylog2016_dataset.zip.1’ saved [4944490674/4944490674]



In [2]:
!unzip -o dailylog2016_dataset.zip -d dailylog_data/

Archive:  dailylog2016_dataset.zip
  inflating: dailylog_data/subject7/images/process_small.png  
 extracting: dailylog_data/subject7/images/preview.png  
  inflating: dailylog_data/subject7/images/process_large.png  
  inflating: dailylog_data/subject7/data/process.pdf  
  inflating: dailylog_data/subject7/data/gps_csv.zip  
  inflating: dailylog_data/subject7/data/ori_csv.zip  
  inflating: dailylog_data/subject7/data/posture.csv  
  inflating: dailylog_data/subject7/data/activity.csv  
  inflating: dailylog_data/subject7/data/acc_csv.zip  
  inflating: dailylog_data/subject7/data/environment.csv  
  inflating: dailylog_data/subject7/data/activity.xes  
  inflating: dailylog_data/subject5/images/process_small.png  
 extracting: dailylog_data/subject5/images/preview.png  
  inflating: dailylog_data/subject5/images/process_large.png  
  inflating: dailylog_data/subject5/data/process.pdf  
  inflating: dailylog_data/subject5/data/posture.csv  
  inflating: dailylog_data/subject5/data/ac

In [4]:
for i in range(1,8):
  !unzip -o dailylog_data/subject$i/data/ori_csv.zip -d dailylog_data/subject$i/data/

Archive:  dailylog_data/subject1/data/ori_csv.zip
  inflating: dailylog_data/subject1/data/SensorOrientationData_labeled_day10.csv  
  inflating: dailylog_data/subject1/data/SensorOrientationData_labeled_day11.csv  
  inflating: dailylog_data/subject1/data/SensorOrientationData_labeled_day12.csv  
  inflating: dailylog_data/subject1/data/SensorOrientationData_labeled_day13.csv  
  inflating: dailylog_data/subject1/data/SensorOrientationData_labeled_day14.csv  
  inflating: dailylog_data/subject1/data/SensorOrientationData_labeled_day1.csv  
  inflating: dailylog_data/subject1/data/SensorOrientationData_labeled_day2.csv  
  inflating: dailylog_data/subject1/data/SensorOrientationData_labeled_day3.csv  
  inflating: dailylog_data/subject1/data/SensorOrientationData_labeled_day4.csv  
  inflating: dailylog_data/subject1/data/SensorOrientationData_labeled_day6.csv  
  inflating: dailylog_data/subject1/data/SensorOrientationData_labeled_day7.csv  
  inflating: dailylog_data/subject1/data/Se

In [40]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import sklearn.preprocessing
import xgboost
from copy import deepcopy

In [25]:
combined_dfs = []
for s in range(1, 8): #for every subject
  dir = 'dailylog_data/subject{0}/data/'.format(s)
  dfs = []
  read_days = 0
  i = 1
  while i <= 13 and read_days <= 1: #for first 1 days of observation
    filename = 'SensorOrientationData_labeled_day{0}.csv'.format(i)
    path = dir + filename
    try:
      df = pd.read_csv(path) #read in data from csv
      df = df.iloc[::5]
      df = df[df.label_valid == True]
      dfs.append(df)
      read_days += 1
    except FileNotFoundError:
      print('Skipping day {0} for subject {1}'.format(i, s))
    i += 1
    
  if len(dfs) > 0:  
    combined = pd.concat(dfs) #concatenate all days into a single dataframe
    combined['subject'] = s #add a column identifying the subject
    combined_dfs.append(combined) #add dataframe to list of combined dataframes

    
    

Skipping day 1 for subject 5
Skipping day 2 for subject 5
Skipping day 3 for subject 5
Skipping day 4 for subject 5
Skipping day 5 for subject 5
Skipping day 6 for subject 5
Skipping day 7 for subject 5
Skipping day 8 for subject 5
Skipping day 9 for subject 5
Skipping day 10 for subject 5
Skipping day 11 for subject 5
Skipping day 12 for subject 5
Skipping day 13 for subject 5


In [27]:
train_df = pd.concat(combined_dfs)

In [31]:
label_enc = sklearn.preprocessing.LabelEncoder()
label_enc.fit(train_df.label_activity.unique())


LabelEncoder()

In [32]:
train_df['activity_target'] = label_enc.transform(train_df.label_activity)

In [35]:
enviro_enc = sklearn.preprocessing.LabelEncoder()
enviro_enc.fit(train_df.label_environment)
train_df['label_environment_enc'] = enviro_enc.transform(train_df.label_environment)

In [38]:
devicepos_enc = sklearn.preprocessing.LabelEncoder()
devicepos_enc.fit(train_df.label_deviceposition)
train_df['label_deviceposition_enc'] = devicepos_enc.transform(train_df.label_deviceposition)

In [39]:
posture_enc = sklearn.preprocessing.LabelEncoder()
posture_enc.fit(train_df.label_posture)
train_df['label_posture_enc'] = posture_enc.transform(train_df.label_posture)

In [45]:
train_df_enc = deepcopy(train_df)
train_df_enc.drop(columns=['label_activity', 'label_environment', 'label_deviceposition', 'label_posture', 'label_valid', 'subject', 'id'], inplace=True)

In [46]:
train_df_enc

,attr_time,attr_azimuth,attr_pitch,attr_roll,activity_target,label_environment_enc,label_deviceposition_enc,label_posture_enc
0,11.03.15 07:58:37.607,247.140260,-24.173317,-5.357847,23,13,9,16
5,11.03.15 07:58:37.706,231.732710,-23.204926,-4.549244,23,13,9,16
10,11.03.15 07:58:37.811,204.415080,-21.849115,-4.396296,23,13,9,16
15,11.03.15 07:58:37.906,195.771030,-20.247486,-4.261816,23,13,9,16
20,11.03.15 07:58:38.013,193.098900,-19.093735,-4.136129,23,13,9,16
...,...,...,...,...,...,...,...,...
2800115,20.03.15 22:37:56.643,30.125560,-53.604668,11.291187,3,12,5,9
2800120,20.03.15 22:37:56.741,29.985853,-53.518852,11.463013,3,12,5,9
2800125,20.03.15 22:37:56.841,29.366241,-53.408855,11.626441,3,12,5,9
2800130,20.03.15 22:37:56.942,29.134687,-53.354440,11.729003,3,12,5,9


In [48]:
from google.colab import drive
drive.mount('/content/datadrive')

Mounted at /content/datadrive


In [49]:
%mkdir /content/datadrive/MyDrive/ML4QS/Assignment3


In [57]:
cols = list(train_df_enc.columns)

In [62]:
cols.remove('activity_target')

In [63]:
cols = ['activity_target'] + cols

In [79]:
train_df.to_csv('/content/datadrive/MyDrive/ML4QS/Assignment3/train_df.csv')

In [82]:
train_df_enc.to_csv('/content/datadrive/MyDrive/ML4QS/Assignment3/train_df_enc.csv')

In [85]:
combined_test_dfs = []
for s in range(1, 8): #for every subject
  dir = 'dailylog_data/subject{0}/data/'.format(s)
  dfs = []
  read_days = 0
  i = 13
  while i > 0 and read_days <= 1: #for first 2 days of observation
    filename = 'SensorOrientationData_labeled_day{0}.csv'.format(i)
    path = dir + filename
    try:
      df = pd.read_csv(path) #read in data from csv
      df = df.iloc[::5]
      df = df[df.label_valid == True]
      dfs.append(df)
      read_days += 1
    except FileNotFoundError:
      print('Skipping day {0} for subject {1}'.format(i, s))
    i -= 1
    
  if len(dfs) > 0:  
    combined = pd.concat(dfs) #concatenate all days into a single dataframe
    combined['subject'] = s #add a column identifying the subject
    combined_test_dfs.append(combined) #add dataframe to list of combined dataframes

test_df = pd.concat(combined_test_dfs)


Skipping day 13 for subject 3
Skipping day 12 for subject 3
Skipping day 11 for subject 3
Skipping day 13 for subject 4
Skipping day 13 for subject 5
Skipping day 12 for subject 5
Skipping day 11 for subject 5
Skipping day 10 for subject 5
Skipping day 9 for subject 5
Skipping day 8 for subject 5
Skipping day 7 for subject 5
Skipping day 6 for subject 5
Skipping day 5 for subject 5
Skipping day 4 for subject 5
Skipping day 3 for subject 5
Skipping day 2 for subject 5
Skipping day 1 for subject 5
Skipping day 13 for subject 6
Skipping day 12 for subject 6
Skipping day 11 for subject 6
Skipping day 10 for subject 6
Skipping day 13 for subject 7
Skipping day 12 for subject 7


In [86]:
def encode_df(df, cols):
  encoders = []
  for col in cols:
    enc = sklearn.preprocessing.LabelEncoder()
    enc.fit(df[col].unique())
    df[col+'_enc'] = enc.transform(df[col])
    encoders.append(enc)
  return df, encoders


In [87]:
test_df_enc, test_encoders = encode_df(test_df, ['label_activity', 'label_environment', 'label_deviceposition', 'label_posture', 'label_valid'])

In [90]:
test_df_enc.drop(columns=['label_activity', 'label_environment', 'label_deviceposition', 'label_posture', 'label_valid'], inplace=True)

In [96]:
test_df.to_csv('/content/datadrive/MyDrive/ML4QS/Assignment3/test_df.csv')

In [1]:
test_df_enc.to_csv('/content/datadrive/MyDrive/ML4QS/Assignment3/test_df_enc.csv')

NameError: ignored